<a href="https://colab.research.google.com/github/ngupta23/medium_articles/blob/main/time_series/pycaret/pycaret_ts_sktime_darts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import darts
except:
  !pip install darts

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
try:
  import pycaret
except:
  !pip install pycaret-ts-alpha

In [ ]:
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from functools import reduce

from darts import TimeSeries
from darts.models import NaiveSeasonal

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

## DARTS Adapter

In [ ]:
from sktime.forecasting.base import BaseForecaster

class _DartsAdapter(BaseForecaster):
    """Base class for interfacing statsmodels forecasting algorithms."""
    # https://github.com/alan-turing-institute/sktime/blob/v0.8.0/extension_templates/forecasting.py
    _tags = {
        "scitype:y": "univariate",  # which y are fine? univariate/multivariate/both
        "univariate-only": True,  # does estimator use the exogeneous X?
        "handles-missing-data": False,  # can estimator handle missing data?
        "y_inner_mtype": "pd.Series",  # which types do _fit, _predict, assume for y?
        "X_inner_mtype": "pd.DataFrame",  # which types do _fit, _predict, assume for X?
        "requires-fh-in-fit": False,  # is forecasting horizon already required in fit?
        "X-y-must-have-same-index": True,  # can estimator handle different X/y index?
        "enforce-index-type": None,  # index type that needs to be enforced in X/y
        "capability:pred_int": False,
    }

    _fitted_param_names = ()
    
    def __init__(self):
        self._forecaster = None
        self._fitted_forecaster = None
        super(_DartsAdapter, self).__init__()
       

    def _fit(self, y, X=None, fh=None):
        """Fit to training data.
        Parameters
        ----------
        y : pd.Series
            Target time series to which to fit the forecaster.
        fh : int, list or np.array, optional (default=None)
            The forecasters horizon with the steps ahead to to predict.
        X : pd.DataFrame, optional (default=None)
            Exogenous variables are ignored
        Returns
        -------
        self : returns an instance of self.
        """
        self._forecaster = self._instantiate_model()

        from darts import TimeSeries
        # Darts needs a datetime index (originally passed as PeriodIndex)
        y.index = y.index.astype('datetime64[ns]') 
        y_ts = TimeSeries.from_series(y)
        self._forecaster.fit(y_ts)
        
        # this should happen last
        self._is_fitted = True

        return self

    def _predict(self, fh=None, X=None, return_pred_int=False, alpha=0.05):
        self.check_is_fitted()
        
        # Temporary hack assuming continuous values without any gap.
        # Will need more work
        h = len(list(fh))
        print(h)
        y = self._forecaster.predict(h).values()
        return y

In [ ]:
class Naive(_DartsAdapter):
    def __init__(
        self,
        K=1,
    ):
      self.K = K
      
      super(Naive, self).__init__()

    def _instantiate_model(self):
        # import inside method to avoid hard dependency
        from darts.models import NaiveSeasonal as _NaiveSeasonal
        return _NaiveSeasonal(K = self.K)


## Get Dataset

In [ ]:
from pycaret.datasets import get_data
y = get_data("airline")
train = y[:-36]
test = y[-36:]
train.index, test.index

Period
1949-01    112.0
1949-02    118.0
1949-03    132.0
1949-04    129.0
1949-05    121.0
Freq: M, Name: Number of airline passengers, dtype: float64

(PeriodIndex(['1949-01', '1949-02', '1949-03', '1949-04', '1949-05', '1949-06',
              '1949-07', '1949-08', '1949-09', '1949-10',
              ...
              '1957-03', '1957-04', '1957-05', '1957-06', '1957-07', '1957-08',
              '1957-09', '1957-10', '1957-11', '1957-12'],
             dtype='period[M]', name='Period', length=108),
 PeriodIndex(['1958-01', '1958-02', '1958-03', '1958-04', '1958-05', '1958-06',
              '1958-07', '1958-08', '1958-09', '1958-10', '1958-11', '1958-12',
              '1959-01', '1959-02', '1959-03', '1959-04', '1959-05', '1959-06',
              '1959-07', '1959-08', '1959-09', '1959-10', '1959-11', '1959-12',
              '1960-01', '1960-02', '1960-03', '1960-04', '1960-05', '1960-06',
              '1960-07', '1960-08', '1960-09', '1960-10', '1960-11', '1960-12'],
             dtype='period[M]', name='Period'))

## Train

In [ ]:
model = Naive()
model.fit(train)

Naive()

## Predict

In [ ]:
#### Preiction works with underlying DARTS model ----
model._forecaster.predict(36).values().T

array([[336., 336., 336., 336., 336., 336., 336., 336., 336., 336., 336.,
        336., 336., 336., 336., 336., 336., 336., 336., 336., 336., 336.,
        336., 336., 336., 336., 336., 336., 336., 336., 336., 336., 336.,
        336., 336., 336.]])

In [ ]:
#### Prediction does not work with sktime adapter ----
predictions = model.predict(fh=np.arange(1, 37))

36


ValueError: ignored

## Backup

In [ ]:
from darts.metrics import mape, mase
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis, plot_hist
from darts.datasets import AirPassengersDataset

In [ ]:
series = AirPassengersDataset().load()
series.plot()

In [ ]:
train, val = series.split_before(pd.Timestamp('19580101'))
train.plot(label='training')
val.plot(label='validation')
plt.legend()

In [ ]:
naive_model = NaiveSeasonal(K=1)
naive_model.fit(train)
naive_forecast = naive_model.predict(36)

series.plot(label='actual')
naive_forecast.plot(label='naive forecast (K=1)')
plt.legend();